# Проект по SQL

Наша компания купила крупный сервис для чтения книг по подписке.  

__Задача__  — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

__Цель__ - сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Доступ к базе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

__Проанализируем таблицы__

In [2]:
books = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Видим id книни, автора, название книг, количество страниц, дата публикации, id издательства

In [3]:
authors = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Столбцы id автора и имя автора

In [4]:
ratings = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Столбцы с id рейтинга, книги, имя пользователя, и рейтингом

In [5]:
reviews = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Столбцы с id обзора, книги, именем пользователя, и текстом обзора.

In [6]:
publishers = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


id издательства, название издательства

__Задача 1__ Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
quantity_of_books = '''SELECT count(book_id)
FROM books
WHERE publication_date::date > '2000-01-01';'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(quantity_of_books), con = con)

,count
0,819


Всего 819 книга была выпущена после 1 января 2000 года

__Задача 2__ Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
average_rating_reviews = '''
SELECT b.book_id,b.title, count(DISTINCT re.text) as quantity_of_reviews, AVG(ra.rating) as average_rating
FROM books as b
LEFT JOIN ratings as ra on ra.book_id = b.book_id
LEFT JOIN reviews as re on re.book_id = b.book_id
GROUP BY 1,2
ORDER BY quantity_of_reviews DESC;'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(average_rating_reviews), con = con)

,book_id,title,quantity_of_reviews,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Всего 1000 книг,книга Twilight (Twilight #1)	 имеет больше всего обзоров 7 и рейтинг 3.66

__Задача 3__  Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
publisher = '''
SELECT pub.publisher, count(distinct b.title) as quantity
FROM publishers as pub
LEFT JOIN books as b on b.publisher_id = pub.publisher_id
WHERE b.num_pages > 50
GROUP BY pub.publisher
ORDER BY quantity DESC
LIMIT 1;'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(publisher), con = con)

,publisher,quantity
0,Penguin Books,42


Больше всего книг толщиной больше 50 страниц выпустило издательство Penguin Books в количестве 42 книги

__Задача 4__  Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
author = '''
SELECT a.author ,AVG(r.rating) as average_rating
FROM author as a
LEFT JOIN books as b on b.author_id = a.author_id
LEFT JOIN ratings as r on r.book_id = b.book_id

GROUP BY a.author
HAVING count(r.book_id) > 50
ORDER BY average_rating DESC
LIMIT 1;'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(author), con = con)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.288462


J.K. Rowling/Mary GrandPré автор с самой высокой средней оценкой книг,учитывая только книги с 50 и более оценками, рейтинг 4.29

__Задача 5__  Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
average_reviews = '''
SELECT AVG(tab.quality)
FROM (SELECT username, count(text) as quality FROM reviews GROUP BY username) as tab
WHERE username in 
(SELECT username
FROM ratings
GROUP BY username
HAVING count(username) > 48);'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(average_reviews), con = con)

,avg
0,24.0


Среднее количество обзоров составляет 24, это те пользователи, которые поставили больше 48 оценок

__Вывод__  
Надо добавлять в базу книги с высоким рейтингом, с достаточным количеством обзоров, не менее 3-4 например, это говорит о популярности книги, например Twilight (Twilight #1).Также надо добавлять книги высокорейтинговых авторов. Например J.K. Rowling/Mary GrandPré. Остальные краткие пояснения я оставил после каждой задачи	